In [ ]:
import pandas as pd
import requests
from tkinter import *
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# # Setup splinter
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Establish URL
url = 'https://coinmarketcap.com/'

In [ ]:
# #Say Hi to the URL
# browser.visit(url)

In [ ]:
# for x in range(1, 42):

#     market = pd.read_html(url)

#     browser.find_by_xpath('//*[@id="__next"]/div/div[2]/div/div/div[3]/div/ul/li[8]/a/svg',wait_time=0.5).first.click()

In [ ]:
#Melting Pot
markets = pd.read_html(url)

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
coins = master['Name'].values
coins

In [ ]:
#Setting Up Drop Down Function
root = Tk()
root.geometry( "200x200" )
def show():
    label.config( text = clicked.get() )
    
options = coins
clicked = StringVar()
clicked.set('Click Me!')

drop = OptionMenu( root, clicked, *options)
drop.pack()

button = Button(root, text = 'Select This Coin?', command =show).pack()

label= Label(root, text = " ")
label.pack()

root.mainloop()

content = clicked.get()
#content
crypto = content
crypto

#Don't Forget to 'x' out of the prompt!

In [ ]:
#Create new URL request
exchange = f'https://www.coindesk.com/price/{crypto}'
print(exchange)

In [ ]:
#Stir the pot
trade = requests.get(exchange)
soup = BeautifulSoup(trade.text, 'html.parser')
soup

In [ ]:
#You Spin My Head Right Round
Title = ''
Price = ''
Change24 = ''
MarketCap = ''
Volume = ''
Low24 = ''
High24 = ''
NetChange = ''
Open24 = ''
AllTimeHigh = ''
TotalSupply= ''
Returns24 = ''
ReturnsYTD = ''
Volatility30Day = ''
TransactionCount24 = ''
AvgTransFee24 = ''
ValueTransferred24 = ''
About = ''
try:
    Title = soup.select('div.coin-title .heading')[0].text
    Price = soup.select('div.coin-info-block .data-definition')[0].text
    Change24 = soup.select('div.coin-info-block .data-definition')[1].text
    MarketCap = soup.select('div.coin-info-block .data-definition')[2].text
    Volume = soup.select('div.coin-info-block .data-definition')[3].text
    Low24 = soup.select('div.coin-info-block .data-definition')[4].text
    High24 = soup.select('div.coin-info-block .data-definition')[5].text
    Open24 = soup.select('div.coin-info-block .data-definition')[6].text
    Open24 = soup.select('div.coin-info-block .data-definition')[7].text 
    AllTimeHigh = soup.select('div.coin-info-block .data-definition')[8].text
    TotalSupply= soup.select('div.coin-info-block .data-definition')[9].text
    Returns24 = soup.select('div.coin-info-block .data-definition')[10].text
    ReturnsYTD = soup.select('div.coin-info-block .data-definition')[11].text
    Volatility30Day = soup.select('div.coin-info-block .data-definition')[12].text
    TransactionCount24 = soup.select('div.coin-info-block .data-definition')[13].text
    AvgTransFee24 = soup.select('div.coin-info-block .data-definition')[14].text
    ValueTransferred24 = soup.select('div.coin-info-block .data-definition')[15].text
    About = soup.select('div.coin-about-text')[0].text
except:
    print('Crypto Not Found')

In [ ]:
#DUMP IT ALL
print(f'---------------{Title}--------------------')
print(f'Price : {Price}')
print(f'24hr % Change : {Change24}')
print(f'Market Cap : {MarketCap}')
print(f'24hr Volume : {Volume}')
print(f'24hr Low : {Low24}')
print(f'24hr High : {High24}')
print(f'Net Change : {NetChange}')
print('---------------Metrics--------------------')
print(f'24hr Open : {Open24}')
print(f'All Time High : {AllTimeHigh}')
print(f'Total Supply : {TotalSupply}')
print(f'24hr Returns : {Returns24}')
print(f'Year To Date Returns : {ReturnsYTD}')
print(f'30 Day Volatility : {Volatility30Day}')
print(f'24hr Transaction Count : {TransactionCount24}')
print(f'24hr Average Transaction Fee {AvgTransFee24}')
print(f'24hr Value Transacted : {ValueTransferred24}')
print('---------------About--------------------')
print(About)
print('---------------Source--------------------')
print(url)